This notebook is only for developing, training, and saving the model. For evaluating the model, check `evaluate.ipynb`.

# Datasets

Run this `gcsfuse` cell if you can't list the folders inside of "/gcs"

In [2]:
!gcsfuse --implicit-dirs "~/gcs"

I1114 05:14:45.771818 2023/11/14 05:14:45.771778 Start gcsfuse/0.42.5 (Go version go1.20.3) for app "" using mount point: /home/jupyter/gcs


When using GCS buckets, use "/gcs" instead of "gs://"

In [3]:
#from sklearn.model_selection import train_test_split

dataset_path = "/home/jupyter/gcs/serena-shsw-datasets"
training_dataset = dataset_path + "/FER-2013/train"
test_dataset = dataset_path + "/FER-2013/test"
validation_dataset = dataset_path + "/FER-2013/valid"

# Split the dataset into training and validation sets
#training_dataset, validation_dataset = train_test_split(training_dataset, test_size=0.2, random_state=42)

# Output directory contents
!echo "Train"
!ls {training_dataset}
!echo "Test"
!ls {test_dataset}
!echo "Valid"
!ls {validation_dataset}

Train
angry  disgust	fear  happy  neutral  sad  surprise
Test
angry  disgust	fear  happy  neutral  sad  surprise
Valid


# Import Library

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from google.cloud import aiplatform

2023-11-14 05:14:49.725723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 05:14:51.178254: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-14 05:14:51.178285: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-14 05:14:54.799207: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

# PRE-PROCESSING DATA

In [5]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
NUM_CLASSES = 7  # Anggap ada 7 kelas emosi wajah

In [6]:
train_data_path = training_dataset
test_data_path = test_dataset

# Create Model

In [7]:
# Gunakan ImageDataGenerator untuk augmentasi data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
    train_data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_dataset = test_datagen.flow_from_directory(
    test_data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [8]:
# Muat model ResNet50 yang sudah dilatih tanpa lapisan teratas
base_model = ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

2023-11-14 05:15:11.512794: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-14 05:15:11.512836: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-14 05:15:11.512861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (emotion-detector-nb): /proc/driver/nvidia/version does not exist
2023-11-14 05:15:11.513846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Bekukan lapisan-lapisan model dasar
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Bangun model
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])


# Data Generating

In [11]:
# Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Latih model
model.fit(train_dataset, epochs=50, validation_data=test_dataset)

Epoch 1/50
  4/898 [..............................] - ETA: 39:41 - loss: 2.5069 - accuracy: 0.1953

KeyboardInterrupt: 

# Saving Model

Vertex AI expects the model artifacts to be saved in `BASE_OUTPUT_DIRECTORY/model/` when you want to train a new version of a model

In [12]:
saved_model_path = dataset_path + "/models/serena-emotion-detector/model"

# Do not uncomment this line, it will be done by setup.sh
#model.save(saved_model_path)

After saving, use `evaluate.ipynb` to evaluate the model after loading the artifacts.